In [1]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import glob
import os
from skimage.transform import resize
import cv2
import itertools
import warnings
warnings.filterwarnings('ignore')
import skimage as sk
sk.__version__

'0.14.0'

In [ ]:
"""res = models.resnet34(pretrained=True)
for param in res.parameters():
        param.requires_grad = False
p = 0
for child in res.children():
    print(child)
    print(p)
    if p >= 5:
        for param in child.parameters():
            param.requires_grad = True
            
    p+=1
    
    
class CNN(nn.Module):
    def __init__(self, original_model):
        super(CNN, self).__init__()
        self.feature_extractor = *list(original_model.children())[:-1]
        self.classifier = nn.Linear(in_features=512,out_features=99,bias=True)
    def forward(x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        x = F.softmax(x,dim=1)
        return x

cnn = CNN(res)
#cnn"""

In [2]:
def create_pair_lists(train_test_path):
    
    class_names = [names for names in glob.glob(train_test_path+"/*")]
    #print(class_names)
    
    a = []
    for names in class_names:
        folder = []
        for folders in glob.glob(names+"/*"):
            folder.append(folders)
        a.append(folder)
        

    paired_folders = itertools.zip_longest(*a)
    
    return list(paired_folders)

pair = create_pair_lists(train_test_path= "Frames/Train")
type(pair) 
#print(list(pair[0]))

list

In [3]:
def im_toarray(im):
    array = plt.imread(im)
    array = resize(array,(224,224,3),anti_aliasing=True)
    return array

    


In [4]:
def create_sets(folder,label):
    X_ = []
    Y_ = []
    for frame in glob.glob(folder+"/*.jpg"):
        im_array = im_toarray(frame)
        X_.append(im_array)
        Y_.append(label)
    
    return np.array(X_),np.array(Y_).reshape(-1,1)
        

In [5]:
def one_hot(array, num_classes):
    return np.squeeze(np.eye(num_classes)[array.reshape(-1)])

In [6]:
def normalize(X):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    X_normalized = (X - mean)/std
    return X_normalized
    

In [7]:
def model():

    model = models.resnet18(pretrained=True)
        
    

    
    """model.classifier = nn.Sequential(nn.Linear(in_features=25088, out_features=4096, bias=True),
                                     nn.ReLU(),
                                     nn.Dropout(p=0.5),
                                     nn.Linear(in_features=4096, out_features=4096, bias=True),
                                     nn.ReLU(),
                                     nn.Dropout(p=0.5),
                                     nn.Linear(in_features=4096, out_features=99, bias=True))"""
    model.fc = nn.Linear(in_features=512, out_features=99, bias=True)
    
    for param in model.parameters():
        param.requires_grad = False

    p = 0
    for child in model.children():

        if p >=4:
            for param in child.parameters():
                param.requires_grad = True
        p += 1
    
    return model

def compute_output(X,model):
    out = F.softmax(model.forward(X),dim=1)
    return out

device = torch.device("cuda:0")
model = model().to(device).float()
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
#params = sum([np.prod(p.size()) for p in model_parameters])       


In [8]:
def create_batches(X,Y,batch_size,shuffle=True):
    l = len(Y)
    index = np.random.permutation(l)
    X = X[index,:,:,:]
    Y = Y[index,:]
    #print(len(Y))
    for i in range(0,Y.shape[0],batch_size):
        yield(X[i:i+batch_size,:,:,:],Y[i:i+batch_size,:])
    
    

    

In [15]:
def train_model(model,nb_epoch,learning_rate,paired_training_list,device,batch_size):
    
    optims = filter(lambda p: p.requires_grad,model.parameters())    
    optimizer = torch.optim.Adam(optims,lr=learning_rate)
    #print(sum([np.prod(p.size()) for p in optims]))  
    num_classes = 99
    for epoch in range(1,nb_epoch+1):
        epoch_loss = 0.0
        for pairs in paired_training_list:
            pair_loss = 0.0
            pair_corrects = 0.0
            pairs = list(pairs)

            label = 0
            k = 0

            for folder in pairs:
                if folder is not None:            
                    X,Y = create_sets(folder,label)
                    if k == 0:
                        #print(folder)
                        #print("here")
                        X_train_batch = X.copy()
                        Y_train_batch = Y.copy()
                    else:
                        X_train_batch = np.concatenate([X_train_batch,X],axis=0)
                        Y_train_batch = np.concatenate([Y_train_batch,Y],axis=0)
                    k += 1
                    
                label += 1
            

            print(X_train_batch.shape,Y_train_batch.shape)
            num_samples = Y_train_batch.shape[0]
            print("num_samples: "+str(num_samples))
            X_train_batch = normalize(X_train_batch)
            sub_batches = create_batches(X_train_batch,Y_train_batch,batch_size)
            for X_minibatch,Y_minibatch in sub_batches:
                Y_minibatch = one_hot(Y_minibatch,num_classes)
                X_minibatch, Y_minibatch = torch.from_numpy(X_minibatch.reshape(X_minibatch.shape[0],3,224,224)).to(device).float(), torch.from_numpy(Y_minibatch).to(device).float()
                output = compute_output(X_minibatch,model)
                loss,correct = evaluate(output,Y_minibatch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pair_loss += loss.item()
                pair_corrects += correct
            
            epoch_loss += pair_loss
            print("pair loss:" +str(pair_loss))
            print("pair accuracy: "+str(100*pair_corrects/num_samples))
            
        print("epoch: "+str(epoch))
        print("epoch loss: " +str(epoch_loss/num_classes))
    
        
            
        

In [18]:
def evaluate(out,real):
    log_loss = torch.mean(-real*torch.log(out))
    mean_loss = torch.mean((out-real)**2)
    
    loss = (mean_loss + log_loss)/2.0
    
    num_of_samples = real.size()[0]
    real_arg = torch.argmax(real,dim=1)
    out_arg = torch.argmax(out,dim=1)
    #print(real_arg,out_arg)
    #print(sum(out_arg==real_arg).item())
    correct = sum(out_arg==real_arg).item()
    return log_loss,correct
    
    
    
    
    

In [ ]:
train_model(model,2,1.e-3,pair,device,32)

(749, 224, 224, 3) (749, 1)
num_samples: 749
pair loss:0.32630000123754144
pair accuracy: 69.02536715620828
(730, 224, 224, 3) (730, 1)
num_samples: 730
pair loss:0.2714096244890243
pair accuracy: 70.68493150684931
(778, 224, 224, 3) (778, 1)
num_samples: 778
pair loss:0.17417144868522882
pair accuracy: 83.16195372750643
(778, 224, 224, 3) (778, 1)
num_samples: 778
pair loss:0.1269367238273844
pair accuracy: 86.63239074550128


In [ ]:
a = np.array([0,1,1,2,3,3,3,4,5,5,5,5])
b = np.array([1,0,0,2,4,3,5,5,4,4,5,1])

a = torch.from_numpy(one_hot(a,7)).float()
b = torch.from_numpy(one_hot(b,7)).float()

loss,accuracy = evaluate(a,b)
print(loss,accuracy)

In [ ]:
"""def call_data(selected_videos,class_path,label):
    X, Y = [], []
    #class_names = [names for names in glob.glob(path+"/*")]
    
    video_folders = [video for video in glob.glob(class_path+"/*")]
    #print(video_folders)
    shuffled_videos = list(np.random.permutation(video_folders))
    unused_videos = list(set(shuffled_videos) - set(selected_videos))
    if len(unused_videos) != 0:
        selected_video = unused_videos[0]
        selected_videos.append(selected_video)
        
        for frame in glob.glob(selected_video+"/*.jpg"):
            im_array = im_toarray(frame)
            X.append(im_array)
            Y.append(label)
    
    return np.array(X),np.array(Y),selected_videos"""

            

In [ ]:
b = np.array([1,1,1,1,1,2,2,4,4,9,9,9])
a = np.array([1,1,1,1,1,2,2,4,4,9,9,9])

on = one_hot(b,10)
#b_n = normalize(b)
#b_n
on

In [ ]:
on_t = torch.from_numpy(on)
print(torch.argmax(on_t,dim=1).topk(5))
arg = np.argmax(on,axis=1)
#on_t = torch.from_numpy(on)
#argt = torch.argmax(on_t,dim=1)
#arg,arg_t
